In [4]:
using Interpolations, FourierTools, Random, Flux, Base.Threads, Distributions, NPZ


function angMuteRange!(gth, angmtrx, m, l)

    nt, nx = size(gth)
    m = sin(deg2rad(m))^2
	l = sin(deg2rad(l))^2
	@threads for ti = 2:nt
		@threads for xi = 1:nx
		   if angmtrx[ti,xi] <= l
				gth[ti,1:xi] .= 0f0
		   end		
           if angmtrx[ti,xi] >= m
				gth[ti,xi:end] .= 0f0
		   end

		end
	end
	return gth
end


function make_angle_stacks(bsize, near_min, near_max, mid_min, mid_max, far_min, far_max)

          count = 0
      @threads for g = 1:bsize
          
          # read in input an target full gathers
		  gthc_inp = npzread("./synthetics/input_full/inp_"*string(g)*".npy")
		  gthc_targ = npzread("./synthetics/target_full/targ_"*string(g)*".npy")

          # make copies of arrays to store angle muted gathers in later
		  gthc_inp_near = copy(gthc_inp)
		  gthc_targ_near = copy(gthc_targ)
		  gthc_inp_mid = copy(gthc_inp)
		  gthc_targ_mid = copy(gthc_targ)
		  gthc_inp_far = copy(gthc_inp)
		  gthc_targ_far = copy(gthc_targ)

          ang2 = npzread("./synthetics/angle_matrices/matrix_"*string(g)*".npy")
          
          # angle mute data into near mid far ranges
		  gthc_input_near = angMuteRange!(gthc_inp_near, ang2, near_max, near_min)
		  gthc_target_near = angMuteRange!(gthc_targ_near, ang2, near_max, near_min)
		  gthc_input_mid = angMuteRange!(gthc_inp_mid, ang2, mid_max, mid_min)
		  gthc_target_mid = angMuteRange!(gthc_targ_mid, ang2, mid_max, mid_min)
		  gthc_input_far = angMuteRange!(gthc_inp_far, ang2, far_max, far_min)
		  gthc_target_far = angMuteRange!(gthc_targ_far, ang2, far_max, far_min)
		  

          # save angle muted gathers in folders
		  npzwrite("./synthetics/input_near/inp_near_"*string(g)*".npy", gthc_input_near)
		  npzwrite("./synthetics/target_near/targ_near_"*string(g)*".npy", gthc_target_near)
		  npzwrite("./synthetics/input_mid/inp_mid_"*string(g)*".npy", gthc_input_mid)
		  npzwrite("./synthetics/target_mid/targ_mid_"*string(g)*".npy", gthc_target_mid)
		  npzwrite("./synthetics/input_far/inp_far_"*string(g)*".npy", gthc_input_far)
		  npzwrite("./synthetics/target_far/targ_far_"*string(g)*".npy", gthc_target_far)
		  
          count = count+1
		  println(count)
		end
	  end

make_angle_stacks (generic function with 1 method)

In [5]:
# check number of threads being used
println(Threads.nthreads())

1


In [6]:
# specify number of gathers
bsize = 3

# specify near mid and far angle ranges
near_min = 0
near_max = 10

mid_min = 10
mid_max = 20

far_min = 20
far_max = 30

make_angle_stacks(bsize, near_min, near_max, mid_min, mid_max, far_min, far_max)

1
2
3
